In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import LassoSelector
from matplotlib.path import Path
from ipywidgets import FileUpload, Button, Output, FloatText
from IPython.display import display

# Enable interactive plots
%matplotlib widget

# Create widgets
upload = FileUpload(accept='.jpg,.png', multiple=False)
button = Button(description="Process Image")
scale_input = FloatText(description="Scale (in):", value=1.0)
measure_button = Button(description="Measure Area")
output = Output()

# Global variables
image = None  # Stores the uploaded image
image_rgb = None  # Stores the RGB image
fig, ax = None, None  # Matplotlib figure and axis
lasso = None  # Lasso tool instance
selected_pixels = []  # Stores selected pixels

def process_image(upload_widget):
    """Reads an uploaded image file and converts it to a NumPy array."""
    if not upload_widget.value:
        print("Error: No image uploaded.")
        return None

    # Extract the first file from the tuple
    file_data = upload_widget.value[0]
    file_bytes = file_data['content']

    # Convert byte content into a NumPy array
    image_array = np.frombuffer(file_bytes, np.uint8)

    # Decode image using OpenCV
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    if img is None:
        print("Error: Failed to decode image.")
        return None

    return img

def on_button_click(b):
    """Handles button click to process the uploaded image."""
    global image, image_rgb, fig, ax, lasso, selected_pixels

    with output:
        output.clear_output()  # Clear previous outputs

        # Process the uploaded image
        image = process_image(upload)

        if image is None:
            print("Please upload a valid image file.")
            return

        # Convert the image from BGR to RGB (for display)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Create a new figure for interactive selection
        fig, ax = plt.subplots()
        ax.imshow(image_rgb)
        ax.set_title("Use Lasso Tool to Select Region")
        ax.axis('off')

        selected_pixels = []  # Reset selection

        # Activate Lasso Selector
        lasso = LassoSelector(ax, on_lasso_select)

        plt.show()

def on_lasso_select(vertices):
    """Handles Lasso Selector events."""
    global image_rgb, selected_pixels

    # Store selected region
    selected_pixels = np.array(vertices, dtype=np.int32)

    # Create a mask from the selected region
    mask = np.zeros(image_rgb.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [selected_pixels], 255)

    # Apply mask to the original image
    selected_region = cv2.bitwise_and(image_rgb, image_rgb, mask=mask)

    # Display the selected region
    fig, ax = plt.subplots()
    ax.imshow(selected_region)
    ax.set_title("Selected Region")
    ax.axis('off')
    plt.show()

def calculate_area():
    """Calculates the area of the selected region in inches² using a reference scale."""
    global selected_pixels

    if not selected_pixels.any():
        print("Error: No region selected. Use the Lasso tool first.")
        return

    # Convert selected polygon to contour
    contour = np.array(selected_pixels, dtype=np.int32)

    # Compute the area in pixels
    pixel_area = cv2.contourArea(contour)

    # Get user-defined scale (pixels per inch)
    scale_inches = scale_input.value  # Real-world inches of reference
    scale_pixels = np.linalg.norm(selected_pixels[0] - selected_pixels[1])  # First 2 points as reference

    if scale_pixels == 0:
        print("Error: Invalid scale selection. Please ensure correct points.")
        return

    pixels_per_inch = scale_pixels / scale_inches

    # Convert area from pixels² to inches²
    area_inches = pixel_area / (pixels_per_inch ** 2)

    print(f"Measured Area: {area_inches:.2f} inches²")

# Display widgets
button.on_click(on_button_click)
measure_button.on_click(lambda b: calculate_area())
display(upload, button, scale_input, measure_button, output)
